# yelp capstone project
Consist of 5 tables e.g Business, Review, User, Tip, Checkin.\
For dataset details kindly [visit](https://www.yelp.com/dataset/documentation/main)\
Bootcamper : Robert, Sining, Theresa, Kamruddin

The project was initialized and capstoned to pass the knowledge and learning into application.

Now we are importing necessary packages 

In [1]:
import json
import pandas as pd
import sqlalchemy
from dotenv import dotenv_values
from dotenv import load_dotenv
import psycopg2

In [2]:
import sql_functions as sf
from sql_functions import get_sql_config, get_data, get_dataframe, get_engine

In [3]:
# add 'data/' to .gitignore so that you do not upload data to github
!echo 'data/' >> .gitignore

Opening json file from business dataset together with unpacking 'arrributes' columns to find to particular hypothetic columns.

In [4]:
data = []

with open('data/yelp_academic_dataset_business.json') as f:

    for line in f:
        
        #Parse the JSON data
        json_dict = json.loads(line)

        if json_dict['attributes']:
            if 'BusinessParking' in json_dict['attributes'].keys():

                json_dict['attributes']['BusinessParking'] = eval(json_dict['attributes']['BusinessParking'])

        data.append(json_dict)

normlize dataset

In [5]:
biz_df = pd.json_normalize(data)

In [6]:
biz_df.shape

(150346, 65)

Now we have 150,346 rows and 65 columns. We need to clean on column level then deep dive into cells.

## Dropping empty category rows and reset index on existing dataframe

In [7]:
# drop row that have no category labels
biz_df.dropna(subset='categories', inplace=True)
# reset index
biz_df.reset_index(drop=True, inplace=True)

In [8]:
# import category labels from CSV and filter dataframe to keep row that have category label Approx 30 min
cat_data = pd.read_csv('data/categories.csv', header=None)
categories = cat_data[0].tolist()
filtered_df = pd.DataFrame()

filtered_df = biz_df.loc[biz_df['categories'].str.contains('|'.join(categories))]

In [9]:
filtered_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.AcceptsInsurance,attributes.BestNights,attributes.BYOB,attributes.Corkage,attributes.BYOBCorkage,attributes.HairSpecializesIn,attributes.Open24Hours,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.DietaryRestrictions
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150224,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150225,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150233,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150236,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66819 entries, 3 to 150237
Data columns (total 65 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   business_id                            66819 non-null  object 
 1   name                                   66819 non-null  object 
 2   address                                66819 non-null  object 
 3   city                                   66819 non-null  object 
 4   state                                  66819 non-null  object 
 5   postal_code                            66819 non-null  object 
 6   latitude                               66819 non-null  float64
 7   longitude                              66819 non-null  float64
 8   stars                                  66819 non-null  float64
 9   review_count                           66819 non-null  int64  
 10  is_open                                66819 non-null  int64  
 11  catego

### Cleaning columns from business table

In [11]:
#1 dropping less important columns
filtered_df.drop(['address','postal_code','hours','is_open','attributes.ByAppointmentOnly', 'attributes.BikeParking','attributes.CoatCheck',
       'attributes.Caters', 'attributes.WiFi', 'attributes.WheelchairAccessible', 'attributes.HappyHour', 'attributes.HasTV', 'attributes.DogsAllowed', 'attributes.GoodForKids',
       'attributes.RestaurantsAttire', 'attributes.GoodForMeal', 'attributes.BusinessAcceptsBitcoin', 'attributes.Smoking','attributes.Music',
       'attributes.GoodForDancing', 'attributes.AcceptsInsurance', 'attributes.BestNights', 'attributes.BYOB', 'attributes.Corkage', 'attributes.BYOBCorkage',
       'attributes.HairSpecializesIn','attributes.AgesAllowed','attributes.DietaryRestrictions','attributes.BusinessParking','attributes', 'attributes.Ambience',
       'attributes.DriveThru', 'attributes.Open24Hours','attributes.RestaurantsCounterService','attributes.RestaurantsTableService','attributes.RestaurantsGoodForGroups'], axis=1, errors='ignore', inplace=True)

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/1147229934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['address','postal_code','hours','is_open','attributes.ByAppointmentOnly', 'attributes.BikeParking','attributes.CoatCheck',


In [12]:
filtered_df.isna().sum()

business_id                                  0
name                                         0
city                                         0
state                                        0
latitude                                     0
longitude                                    0
stars                                        0
review_count                                 0
categories                                   0
attributes.BusinessAcceptsCreditCards     8735
hours.Monday                             17335
hours.Tuesday                            14127
hours.Wednesday                          11737
hours.Thursday                           10755
hours.Friday                             10387
hours.Saturday                           11658
attributes.RestaurantsPriceRange2        10370
attributes.RestaurantsTakeOut             8896
attributes.RestaurantsDelivery           12934
attributes.BusinessParking.garage        11326
attributes.BusinessParking.street        12699
attributes.Bu

In [13]:
filtered_df.columns

Index(['business_id', 'name', 'city', 'state', 'latitude', 'longitude',
       'stars', 'review_count', 'categories',
       'attributes.BusinessAcceptsCreditCards', 'hours.Monday',
       'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday',
       'hours.Saturday', 'attributes.RestaurantsPriceRange2',
       'attributes.RestaurantsTakeOut', 'attributes.RestaurantsDelivery',
       'attributes.BusinessParking.garage',
       'attributes.BusinessParking.street',
       'attributes.BusinessParking.validated',
       'attributes.BusinessParking.lot', 'attributes.BusinessParking.valet',
       'attributes.OutdoorSeating', 'attributes.RestaurantsReservations',
       'hours.Sunday', 'attributes.Alcohol', 'attributes.NoiseLevel'],
      dtype='object')

In [14]:
#2 renaming the header
filtered_df.rename(columns={'attributes.BusinessAcceptsCreditCards': 'accepts_credit_cards','hours.Monday':'monday', 'hours.Tuesday':'tuesday','hours.Wednesday':'wednesday',
                       'hours.Thursday':'thursday','hours.Friday':'friday','hours.Saturday':'saturday','hours.Sunday':'sunday','attributes.RestaurantsPriceRange2':'price_range',
                       'attributes.RestaurantsTakeOut':'take_out','attributes.RestaurantsDelivery':'delivery','attributes.BusinessParking.garage':'parking_garage',
                       'attributes.BusinessParking.street':'parking_street','attributes.BusinessParking.validated':'parking_validated','attributes.BusinessParking.lot':'parking_lot',
                       'attributes.BusinessParking.valet':'parking_valet','attributes.OutdoorSeating':'outdoor_seating','attributes.RestaurantsReservations':'seats_reservations',
                       'attributes.Alcohol':'alcohol','attributes.Ambience':'ambience','attributes.RestaurantsTableService':'table_service','attributes.RestaurantsGoodForGroups':'for_groups',
                       'attributes.DriveThru':'drive_thru','attributes.Open24Hours':'open_24_hours','attributes.RestaurantsCounterService':'counter_service','attributes.NoiseLevel': 'noise_level'}, inplace=True)

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/945557163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'attributes.BusinessAcceptsCreditCards': 'accepts_credit_cards','hours.Monday':'monday', 'hours.Tuesday':'tuesday','hours.Wednesday':'wednesday',


In [15]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66819 entries, 3 to 150237
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   business_id           66819 non-null  object 
 1   name                  66819 non-null  object 
 2   city                  66819 non-null  object 
 3   state                 66819 non-null  object 
 4   latitude              66819 non-null  float64
 5   longitude             66819 non-null  float64
 6   stars                 66819 non-null  float64
 7   review_count          66819 non-null  int64  
 8   categories            66819 non-null  object 
 9   accepts_credit_cards  58084 non-null  object 
 10  monday                49484 non-null  object 
 11  tuesday               52692 non-null  object 
 12  wednesday             55082 non-null  object 
 13  thursday              56064 non-null  object 
 14  friday                56432 non-null  object 
 15  saturday              5

In [16]:
filtered_df.columns

Index(['business_id', 'name', 'city', 'state', 'latitude', 'longitude',
       'stars', 'review_count', 'categories', 'accepts_credit_cards', 'monday',
       'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'price_range',
       'take_out', 'delivery', 'parking_garage', 'parking_street',
       'parking_validated', 'parking_lot', 'parking_valet', 'outdoor_seating',
       'seats_reservations', 'sunday', 'alcohol', 'noise_level'],
      dtype='object')

### Cleaning data inside the cells

In [17]:
filtered_df['alcohol'].unique()

array(["u'none'", nan, "u'full_bar'", "'none'", "'full_bar'",
       "u'beer_and_wine'", "'beer_and_wine'", 'None'], dtype=object)

In [18]:
filtered_df['noise_level'].unique()

array([nan, "u'average'", "u'quiet'", "'average'", "u'loud'",
       "u'very_loud'", "'quiet'", "'very_loud'", "'loud'", 'None'],
      dtype=object)

In [19]:
filtered_df['alcohol'] = filtered_df['alcohol'].str.replace("u'", "").str.replace("'", "")
filtered_df['noise_level'] = filtered_df['noise_level'].str.replace("u'", "").str.replace("'", "")

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/3781653990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['alcohol'] = filtered_df['alcohol'].str.replace("u'", "").str.replace("'", "")
/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/3781653990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['noise_level'] = filtered_df['noise_level'].str.replace("u'", "").str.replace("'", "")


In [20]:
filtered_df['alcohol'].unique()

array(['none', nan, 'full_bar', 'beer_and_wine', 'None'], dtype=object)

In [21]:
filtered_df['noise_level'].unique()

array([nan, 'average', 'quiet', 'loud', 'very_loud', 'None'], dtype=object)

In [22]:
# creating function to check the Values either true in any case
def parking_check(row):
    if row['parking_garage'] == True or row['parking_street'] == True or row['parking_validated'] == True or row['parking_lot'] == True or row['parking_valet'] == True:
        return True
    else:
        return False

In [23]:
# running parking check by row if anywhere found True returns True in a new Column else all returning False.
filtered_df['parking_available'] = filtered_df.apply(parking_check, axis=1)

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/3692541997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['parking_available'] = filtered_df.apply(parking_check, axis=1)


In [24]:
filtered_df

,business_id,name,city,state,latitude,longitude,stars,review_count,categories,accepts_credit_cards,...,parking_street,parking_validated,parking_lot,parking_valet,outdoor_seating,seats_reservations,sunday,alcohol,noise_level,parking_available
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",False,...,True,False,False,False,False,NaN,7:0-21:0,none,NaN,True
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,40.338183,-75.471659,4.5,13,"Brewpubs, Breweries, Food",True,...,None,None,True,False,NaN,NaN,12:0-18:0,NaN,NaN,True
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",True,...,NaN,NaN,NaN,NaN,True,False,8:0-22:0,none,NaN,False
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",True,...,False,False,True,False,True,False,NaN,full_bar,average,True
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",True,...,False,False,False,False,True,False,6:0-21:0,none,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150224,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,43.615401,-116.284689,4.0,33,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",True,...,False,False,True,False,True,False,0:0-0:0,none,loud,True
150225,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,Oro Valley,AZ,32.409552,-110.943073,4.5,41,"Specialty Food, Food, Coffee & Tea, Coffee Roa...",True,...,False,False,True,False,True,NaN,6:0-15:0,NaN,NaN,True
150233,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,PA,39.935982,-75.158665,4.5,35,"Restaurants, Mexican",True,...,True,None,False,False,False,False,11:0-22:0,none,NaN,True
150236,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,DE,39.856185,-75.427725,4.5,14,"Restaurants, Comfort Food, Food, Food Trucks, ...",True,...,True,False,False,False,NaN,NaN,0:0-0:0,none,NaN,True


In [25]:
# dropping all the parking extensions from the table to be in more cleaned way
filtered_df.drop(['parking_garage', 'parking_street','parking_validated','parking_lot','parking_valet'], axis=1, inplace=True)

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_7356/1671550035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['parking_garage', 'parking_street','parking_validated','parking_lot','parking_valet'], axis=1, inplace=True)


In [26]:
filtered_df

,business_id,name,city,state,latitude,longitude,stars,review_count,categories,accepts_credit_cards,...,saturday,price_range,take_out,delivery,outdoor_seating,seats_reservations,sunday,alcohol,noise_level,parking_available
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",False,...,7:0-21:0,1,True,False,False,NaN,7:0-21:0,none,NaN,True
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,40.338183,-75.471659,4.5,13,"Brewpubs, Breweries, Food",True,...,12:0-22:0,NaN,True,NaN,NaN,NaN,12:0-18:0,NaN,NaN,True
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,36.269593,-87.058943,2.0,6,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",True,...,9:0-22:0,1,True,True,True,False,8:0-22:0,none,NaN,False
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,38.565165,-90.321087,3.0,19,"Pubs, Restaurants, Italian, Bars, American (Tr...",True,...,NaN,1,True,False,True,False,NaN,full_bar,average,True
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,36.208102,-86.768170,1.5,10,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",True,...,6:0-17:0,1,True,True,True,False,6:0-21:0,none,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150224,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,43.615401,-116.284689,4.0,33,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",True,...,0:0-0:0,1,False,False,True,False,0:0-0:0,none,loud,True
150225,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,Oro Valley,AZ,32.409552,-110.943073,4.5,41,"Specialty Food, Food, Coffee & Tea, Coffee Roa...",True,...,6:0-15:0,2,True,False,True,NaN,6:0-15:0,NaN,NaN,True
150233,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,PA,39.935982,-75.158665,4.5,35,"Restaurants, Mexican",True,...,11:0-2:0,2,True,True,False,False,11:0-22:0,none,NaN,True
150236,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,DE,39.856185,-75.427725,4.5,14,"Restaurants, Comfort Food, Food, Food Trucks, ...",True,...,0:0-0:0,1,True,False,NaN,NaN,0:0-0:0,none,NaN,True


### Now we have 66,819 rows and 25 analysable columns for further process.
### Calling review date and filter selected business_id

In [24]:
filtered_df['business_id'].nunique()

66819

## Filtering out dataset by state PA ######## (Execute Next 4 steps ) 

In [27]:
#1 create function for filtering by state

def state_filter(dataframe, column, state):
    businesses_new = dataframe[dataframe[column] == state].copy()
    return businesses_new

In [28]:
#2 filter for all businesses in Pennsylvania 

filtered_df_pa = state_filter(filtered_df, 'state', 'PA')

In [29]:
#3 delete the columns 'city', 'state', 

filtered_df_pa.drop(columns=['city', 'state'], inplace=True, errors='ignore')

Then the file will narrow down to 15,830 rows from 66,819 rows.

In [30]:
filtered_df_pa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15830 entries, 3 to 150233
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   business_id           15830 non-null  object 
 1   name                  15830 non-null  object 
 2   latitude              15830 non-null  float64
 3   longitude             15830 non-null  float64
 4   stars                 15830 non-null  float64
 5   review_count          15830 non-null  int64  
 6   categories            15830 non-null  object 
 7   accepts_credit_cards  14370 non-null  object 
 8   monday                11136 non-null  object 
 9   tuesday               12051 non-null  object 
 10  wednesday             12766 non-null  object 
 11  thursday              13017 non-null  object 
 12  friday                13103 non-null  object 
 13  saturday              12828 non-null  object 
 14  price_range           13419 non-null  object 
 15  take_out              1

In [31]:
#4 lookup business_id into review file in a chunk and appending in df then writing into csv Approx 1m 2s
filtered = filtered_df_pa['business_id'].to_list()
column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
chunks = pd.read_json('data/y_review.json', lines=True, chunksize=100000)

with open('data/reviews_pa.csv', 'w') as f:
    header = ','.join(column_names)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['business_id'].isin(filtered)].to_csv(f, mode='a', header=False, index=False)

# Push to SQL

In [32]:
# now reading the file chunked into csv Approx 2.15 Min
review_df = pd.read_csv('data/reviews_pa.csv')
review_df.shape

(1205562, 9)

In [33]:
# checking counts of business_id in review table
print(f"Number of distinct Business id: {review_df['business_id'].nunique()}")
print(f"Number of Business id: {review_df['business_id'].count()}")

Number of distinct Business id: 15830
Number of Business id: 1205562


In [34]:
# checking counts of user_id in review table
print(f"Number of distinct user id: {review_df['user_id'].nunique()}")
print(f"Number of user id: {review_df['user_id'].count()}")

Number of distinct user id: 327497
Number of user id: 1205562


In [54]:
review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
2,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
3,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16
4,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
...,...,...,...,...,...,...,...,...,...
1205557,MVg4YUQeEhCA7Z7RsBJSVg,7-7A0Avj47slLGV7yBFc8w,ytynqOUb3hjKeJfRj5Tshw,3,1,0,0,"I was so excited about all the food I saw, but...",2013-07-25 21:00:15
1205558,nLjbVsETpqO17RbFcqskkA,am7-gkH_PDz598oTdYSD6A,3gVSrS4kffGGZT8oXHsIcw,3,2,0,2,"*Later Yelp* I've only been here once, but I l...",2014-11-03 14:45:46
1205559,KlHxcAifUF5zDKpJCBrRsw,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4,1,0,0,"ice cream! ice cream sodas, sundaes!! \n\nwant...",2012-10-21 04:08:40
1205560,cACxcUY_AIsQKkpDRXuqnw,MCzlzlOw7IGbRAKVjJBPtg,fcGexL5VH5G2Xw0tRj9uOQ,3,1,1,0,This is a good pizza option - they deliver thr...,2018-03-13 13:54:48


In [58]:
review_df['dated'] = pd.to_datetime(review_df['date']).dt.date

In [61]:
review_df.drop(['date'], axis=1, inplace=True)

In [66]:
review_df.rename({'dated':'date'}, axis=1, inplace=True)

In [67]:
review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04
2,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23
3,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07
4,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14
...,...,...,...,...,...,...,...,...,...
1205557,MVg4YUQeEhCA7Z7RsBJSVg,7-7A0Avj47slLGV7yBFc8w,ytynqOUb3hjKeJfRj5Tshw,3,1,0,0,"I was so excited about all the food I saw, but...",2013-07-25
1205558,nLjbVsETpqO17RbFcqskkA,am7-gkH_PDz598oTdYSD6A,3gVSrS4kffGGZT8oXHsIcw,3,2,0,2,"*Later Yelp* I've only been here once, but I l...",2014-11-03
1205559,KlHxcAifUF5zDKpJCBrRsw,7ziWZULyiZv2TesYNMFf4g,qQO7ErS_RAN4Vs1uX0L55Q,4,1,0,0,"ice cream! ice cream sodas, sundaes!! \n\nwant...",2012-10-21
1205560,cACxcUY_AIsQKkpDRXuqnw,MCzlzlOw7IGbRAKVjJBPtg,fcGexL5VH5G2Xw0tRj9uOQ,3,1,1,0,This is a good pizza option - they deliver thr...,2018-03-13


In [68]:
review_df['user_id'].unique()

array(['mh_-eMZ6K5RLWhZyISBhwA', '_7bHUi9Uuf5__HHc_Q8guQ',
       'eUta8W_HdHMXPzLBBZhL1A', ..., 'Q5l9zBUj9LtGYNWuEEueBQ',
       'Lt7xSv-7QIhKrctrXEwvxA', '7RAI4vRgimfsCfpjyVKD6w'], dtype=object)

In [36]:
# write review table to server 

load_dotenv()

engine = get_engine()

schema = 'hh_analytics_23_3' 
table_name = 'yelp_review_pa' 

# Write records stored in a dataframe to SQL database
if engine!=None:
    try: ## if something goes wrong
        review_df.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error: 
        print(error)
        engine = None
else:
    print('No engine')

(psycopg2.errors.InsufficientPrivilege) must be owner of table yelp_review_pa

[SQL: 
DROP TABLE hh_analytics_23_3.yelp_review_pa]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [76]:
# write business table to server

table_name_2 = 'yelp_business_pa' 

# Write records stored in a dataframe to SQL database
if engine!=None:
    try: ## if something goes wrong
        filtered_df_pa.to_sql(name=table_name_2, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=10000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name_2} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error: 
        print(error)
        engine = None
else:
    print('No engine')

The yelp_business_pa table was imported successfully.


In [34]:
# checking counts of user_id in review table
print(f"Number of distinct user id: {review_df['user_id'].nunique()}")
print(f"Number of user id: {review_df['user_id'].count()}")

Number of distinct user id: 327497
Number of user id: 1205562


In [37]:
review_df['user_id'].unique()

array(['mh_-eMZ6K5RLWhZyISBhwA', '_7bHUi9Uuf5__HHc_Q8guQ',
       'eUta8W_HdHMXPzLBBZhL1A', ..., 'Q5l9zBUj9LtGYNWuEEueBQ',
       'Lt7xSv-7QIhKrctrXEwvxA', '7RAI4vRgimfsCfpjyVKD6w'], dtype=object)

In [38]:
# now chunking the user table from distinct user id's of review table. Approx 1m 25s
filtered_user = review_df['user_id'].unique()
column_user = ['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos']
chunks = pd.read_json('data/y_user.json', lines=True, chunksize=1000)

with open('data/user_pa.csv', 'w') as f:
    header = ','.join(column_user)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['user_id'].isin(filtered_user)].to_csv(f, mode='a', header=False, index=False)

In [39]:
# reading review df
user_df = pd.read_csv('data/user_pa.csv')
user_df.shape

(327496, 22)

In [40]:
user_df

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,191,361,147,1212,5696,2543,2543,815,323
3,AUi8MPWJ0mLkMfwbui27lg,John,109,2010-01-07 18:32:04,154,20,23,NaN,"gy5fWeSv3Gamuq9Ox4MV4g, lMr3LWU6kPFLTmCpDkACxg...",4,...,0,0,0,0,1,6,3,3,0,0
4,1McG5Rn_UDkmlkZOrsdptg,Teresa,7,2009-05-26 16:11:11,18,3,13,NaN,"piejMEdRkGB7-1aL4lL5NQ, X0zFOU6iG95-feQKOXkgrA...",1,...,0,0,0,0,1,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327491,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
327492,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
327493,5GJYbsi6PMLxoRIGh8qE1w,Erin,1,2018-04-16 13:07:20,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
327494,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
user_df.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_writer', 'compliment_photos',
       'comp_count'],
      dtype='object')

In [41]:
user_df.drop(['compliment_funny'],axis=1, inplace=True)

In [42]:
user_df['comp_count'] = user_df['compliment_hot'] + user_df['compliment_more'] + user_df['compliment_profile'] + user_df['compliment_cute'] + user_df['compliment_list'] + user_df['compliment_note'] + user_df['compliment_plain'] + user_df['compliment_cool']  + user_df['compliment_writer'] + user_df['compliment_photos']

In [44]:
user_df.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_writer', 'compliment_photos',
       'comp_count'],
      dtype='object')

In [45]:
user_df.drop(['compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain','compliment_cool', 'compliment_writer', 'compliment_photos',],axis=1, inplace=True)

In [46]:
# creating function to count 0 if blank string elif no comma in string count as one else count of commas + 1.
def count_commas(string):
  if string == "":
    return 0
  else:
    return string.count(",") + 1

In [47]:
user_df['friends_count'] = count_commas(user_df.friends.str)
user_df.sort_values('friends_count', ascending=False, na_position='last')

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,comp_count,friends_count
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,2406,14995.0
52575,iLjMdZi0Tm7DQxX1C1_2dg,Ruggy,2434,2007-07-31 14:47:00,34949,26293,28216,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","KO-pvAqifK7yzuqI9kVaIQ, A39xX4wg4WfwZ75qHThG3A...",2547,3.98,21939,12395.0
275,ZIOCmdFaMIF56FR-nWr_2A,Randy,3315,2011-01-07 15:20:32,48205,27057,38093,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2...","eLOliEa71F4o0P39lWuk9g, y3IH1jtMxGkSvZYnCnh5Yg...",1124,3.77,8200,11026.0
856,Oi1qbcz2m2SnwUeztGYcnQ,Steven,1371,2010-03-19 16:40:37,29572,13919,23028,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","KrIL3TIOJI-tjvU6BlcA-g, ShwO70X2AWTIAvgi9ZL1Ww...",739,3.62,5996,10072.0
399,F_5_UNX-wrAFCXuAkBZRDw,Rodney,1237,2008-11-17 06:15:22,14691,8604,12545,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","q3A2T4SjVtWiTyWCzCWGXA, wC_IBhyEiFmPjUf9hMUjMg...",659,4.09,12167,8809.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
327491,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,NaN,NaN,0,2.83,0,NaN
327492,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,NaN,NaN,0,5.00,0,NaN
327493,5GJYbsi6PMLxoRIGh8qE1w,Erin,1,2018-04-16 13:07:20,0,0,0,NaN,NaN,0,4.00,0,NaN
327494,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,4.92,0,NaN


In [48]:
user_df['elite_count'] = count_commas(user_df.elite.str)
user_df.sort_values('elite_count', ascending=False, na_position='last')

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,comp_count,friends_count,elite_count
1831,hcZqq-a16ZTjaM2p2MljTg,Vicky,1059,2005-12-28 16:38:47,3372,1343,2480,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","Bq9S6wfZ-mal34lMCajC1w, 91sdYRRNbLiF8TmQXX0THg...",180,4.15,857,971.0,17.0
228958,-eVlHbM2KRZQnBqfTXtDvw,Joc,759,2005-09-03 22:07:29,1096,1479,839,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","y_FgKscBFsR4Xf5vIC3zvw, DTWtZJAIPMrZJwUJV0DJmQ...",55,3.77,419,225.0,17.0
89579,5DgFmyjW6hkBtXtTMKl4tA,Lori,711,2006-09-05 17:48:38,2469,1489,1870,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","D7j17PN4IXCNBJc5O_fTdQ, zridJkQiCy5b252gdmHE1A...",178,3.69,2487,763.0,17.0
656,wjClsIuhEjZN2OlkoFPKnQ,Krista,1248,2006-04-25 16:44:03,2235,666,1300,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","3n_QYiWlZdt5C9ogbsLEdA, ikdLkEDRE1vV5oQVPC2f2Q...",110,3.90,368,891.0,17.0
764,nnImk681KaRqUVHlSfZjGQ,Carlos,1107,2006-07-18 20:46:04,4887,1663,3165,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","uQAqbM07zSTRhZcZJyGYrQ, 2l-PyhuEq1U44oiVwdQmVQ...",259,3.96,1153,4214.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327491,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,NaN,NaN,0,2.83,0,NaN,NaN
327492,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,NaN,NaN,0,5.00,0,NaN,NaN
327493,5GJYbsi6PMLxoRIGh8qE1w,Erin,1,2018-04-16 13:07:20,0,0,0,NaN,NaN,0,4.00,0,NaN,NaN
327494,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,4.92,0,NaN,NaN


In [49]:
user_df.drop(['elite','friends','yelping_since'], axis=1, errors='ignore', inplace=True)

In [50]:
user_df['elite_count'].sort_values(ascending=False).unique()

array([17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
        4.,  3.,  2.,  1., nan])

In [51]:
user_df

,user_id,name,review_count,useful,funny,cool,fans,average_stars,comp_count,friends_count,elite_count
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,1259,5994,267,3.91,2406,14995.0,1.0
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,43091,13066,27281,3138,3.74,17500,4646.0,14.0
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,14953,9940,11211,1357,3.85,13164,5843.0,9.0
3,AUi8MPWJ0mLkMfwbui27lg,John,109,154,20,23,4,3.40,10,64.0,NaN
4,1McG5Rn_UDkmlkZOrsdptg,Teresa,7,18,3,13,1,4.29,4,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
327491,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,1,0,0,0,2.83,0,NaN,NaN
327492,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,0,0,0,0,5.00,0,NaN,NaN
327493,5GJYbsi6PMLxoRIGh8qE1w,Erin,1,0,0,0,0,4.00,0,NaN,NaN
327494,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,7,0,0,0,4.92,0,NaN,NaN


# Push to SQL

In [77]:
# write to server

load_dotenv()

engine = get_engine()

schema = 'hh_analytics_23_3' 
table_name_1 = 'yelp_user' 

# Write records stored in a dataframe to SQL database
if engine!=None:
    try: ## if something goes wrong
        user_df.to_sql(name=table_name_1, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name_1} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error: 
        print(error)
        engine = None
else:
    print('No engine')

The yelp_user table was imported successfully.


In [50]:
user_df['elite_count'].sort_values(ascending=False).unique()

array([17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
        4.,  3.,  2.,  1., nan])

In [51]:
pd.set_option('float_format', lambda x: '%.2f' % x)
user_df.describe().T

,count,mean,std,min,25%,50%,75%,max
review_count,327496.00,34.98,134.58,0.00,2.00,7.00,21.00,17473.00
useful,327496.00,73.84,1091.32,0.00,1.00,4.00,17.00,206296.00
funny,327496.00,31.27,726.43,0.00,0.00,1.00,4.00,185823.00
cool,327496.00,44.21,961.49,0.00,0.00,1.00,4.00,199878.00
fans,327496.00,2.58,33.40,0.00,0.00,0.00,0.00,12497.00
average_stars,327496.00,3.64,1.09,1.00,3.00,3.84,4.43,5.00
comp_hot,327496.00,3.22,112.29,0.00,0.00,0.00,0.00,25784.00
comp_more,327496.00,0.49,13.81,0.00,0.00,0.00,0.00,4347.00
comp_profile,327496.00,0.33,19.74,0.00,0.00,0.00,0.00,7039.00
comp_cute,327496.00,0.23,8.87,0.00,0.00,0.00,0.00,2974.00


# For #### total data to run on review json ####

In [52]:
# lookup business_id into review file in a chunk and appending in df then writing into csv Approx 2m 22s
filtered = filtered_df['business_id'].to_list()
column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
chunks = pd.read_json('data/y_review.json', lines=True, chunksize=1000)

with open('data/review_filter_cat.csv', 'w') as f:
    header = ','.join(column_names)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['business_id'].isin(filtered)].to_csv(f, mode='a', header=False, index=False)

In [53]:
# now reading the file chunked into csv Approx 2.15 Min
review_df = pd.read_csv('data/review_filter_cat.csv')
review_df.shape

(5222906, 9)

In [54]:
# checking counts of business_id in review table
print(f"Number of distinct Business id: {review_df['business_id'].nunique()}")
print(f"Number of Business id: {review_df['business_id'].count()}")

Number of distinct Business id: 66819
Number of Business id: 5222904


In [55]:
# checking counts of user_id in review table
print(f"Number of distinct user id: {review_df['user_id'].nunique()}")
print(f"Number of user id: {review_df['user_id'].count()}")

Number of distinct user id: 1522487
Number of user id: 5222905


In [56]:
# checking counts of user_id in review table
print(f"Number of distinct user id: {review_df['user_id'].nunique()}")
print(f"Number of user id: {review_df['user_id'].count()}")

Number of distinct user id: 1522487
Number of user id: 5222905


In [57]:
review_df['user_id'].unique()

array(['mh_-eMZ6K5RLWhZyISBhwA', '8g_iMtfSiwikVnbP2etR0A',
       '_7bHUi9Uuf5__HHc_Q8guQ', ..., 'lZ7bL5W2bPqiULwY0n_skw',
       '6YRmtPVvnD_mexQifxAJsg', '3kJ4ktaXV-RVm49M-y1uTg'], dtype=object)

In [ ]:
# now chunking the user table from distinct user id's of review table. Approx 
filtered_user = review_df['user_id'].unique()
column_user = ['user_id', 'name', 'review_count', 'yelping_since', 'friends', 'useful', 'funny', 'cool', 'fans', 'elite', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos']
chunks = pd.read_json('data/y_user.json', lines=True, chunksize=1000)

with open('data/user_filters.csv', 'w') as f:
    header = ','.join(column_user)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['user_id'].isin(filtered_user)].to_csv(f, mode='a', header=False, index=False)

In [41]:
# reading review df
user_df = pd.read_csv('data/user_filters.csv')
user_df.shape

/var/folders/86/gt6_0hjj7c345vw90_3wrjnm0000gn/T/ipykernel_10404/1627890526.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv('data/user_filters.csv')


(1522476, 22)

In [43]:
# Renaming the headers of dataframe
user_df.rename(columns={'attributes.BusinessAcceptsCreditCards': 'accepts_credit_cards','compliment_hot':'c_hot','compliment_more':'c_more','compliment_profile':'c_more','compliment_cute':'c_cute','compliment_list':'c_list','compliment_note':'c_note','compliment_plain':'c_plain','compliment_cool':'c_cool','compliment_funny':'c_funny','compliment_writer':'c_writer','compliment_photos':'c_photos'}, inplace=True)

In [44]:
user_df

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,c_more,c_more,c_cute,c_list,c_note,c_plain,c_cool,c_funny,c_writer,c_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,NaN,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522471,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1522472,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1522473,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1522474,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# creating function to count 0 if blank string elif no comma in string count as one else count of commas + 1.
def count_commas(string):
  if string == "":
    return 0
  else:
    return string.count(",") + 1

In [60]:
# now counting fans from 'fans' column and adding into a new column as 'count_fans'
user_df['count_fans'] = count_commas(user_df.fans.str)
user_df.sort_values('count_fans', ascending=False, na_position='last')

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,c_cute,c_list,c_note,c_plain,c_cool,c_funny,c_writer,c_photos,count_fans,cool_year
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,56,18,232,844,467,467,239,180,14995.000,1.000
183700,iLjMdZi0Tm7DQxX1C1_2dg,Ruggy,2434,2007-07-31 14:47:00,34949,26293,28216,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","KO-pvAqifK7yzuqI9kVaIQ, A39xX4wg4WfwZ75qHThG3A...",2547,...,341,118,2282,8989,5080,5080,1360,601,12395.000,15.000
714,ZIOCmdFaMIF56FR-nWr_2A,Randy,3315,2011-01-07 15:20:32,48205,27057,38093,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2...","eLOliEa71F4o0P39lWuk9g, y3IH1jtMxGkSvZYnCnh5Yg...",1124,...,25,92,605,2231,1931,1931,1535,563,11026.000,12.000
184044,mV4lknblF-zOKSF8nlGqDA,Scott,789,2010-03-02 17:59:12,4875,2399,3713,"2013,2014,2015,2016,2017,2018,2019,2021","iL6zTnqpPX5lgyLnW_qAEg, yKx4EVaByZzHEqtJfB7RDg...",283,...,18,4,534,1426,1546,1546,238,69,10366.000,8.000
2275,Oi1qbcz2m2SnwUeztGYcnQ,Steven,1371,2010-03-19 16:40:37,29572,13919,23028,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","KrIL3TIOJI-tjvU6BlcA-g, ShwO70X2AWTIAvgi9ZL1Ww...",739,...,8,11,585,2260,1423,1423,654,379,10072.000,13.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522471,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522472,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522473,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522474,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [61]:
# Adding cool year's count from 'cool' column into a new column as 'cool_year'
user_df['cool_year'] = count_commas(user_df.cool.str)
user_df.sort_values('cool_year', ascending=False, na_position='last')

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,c_cute,c_list,c_note,c_plain,c_cool,c_funny,c_writer,c_photos,count_fans,cool_year
3670,gfQqQYI5_hCAGEHlHXIz2Q,Cherie,2113,2005-10-02 00:49:15,5669,1711,3999,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","3phA1Ao38rweA2Ao_fY4sQ, uNynZMWl1-kwadn2zhRbYw...",212,...,11,10,177,418,385,385,112,15,529.000,17.000
183865,53bZ_EsXH71L7iFs5MP9_w,Cynthia,2052,2006-02-10 00:25:28,7713,4098,5173,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","VRGN0zaFkRWlkd5lmkj18g, Nnnc3F4an4GkBdQOGL4NNQ...",396,...,52,45,182,298,646,646,212,195,629.000,17.000
2033,nnImk681KaRqUVHlSfZjGQ,Carlos,1107,2006-07-18 20:46:04,4887,1663,3165,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","uQAqbM07zSTRhZcZJyGYrQ, 2l-PyhuEq1U44oiVwdQmVQ...",259,...,7,15,160,364,242,242,141,60,4214.000,17.000
72,VHdY6oG2JPVNjihWhOooAQ,Jessica,2101,2005-07-21 01:16:04,47831,28830,44899,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","YTqvt6PKmQS-Ygk_JeyXZQ, NhAJ4Fv4xDkWJYo85Y3yOQ...",2627,...,963,153,2124,16674,7324,7324,2313,548,7342.000,17.000
1093,JnGtgOPpkjyWOvWM0SYEXg,Hen,3614,2006-03-29 18:19:32,20463,11998,17330,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","TGCpV0H_0aimzrCWoTDrIA, 1unQbxCkgwrZMRmox79LCg...",498,...,217,73,730,1667,2346,2346,1107,2145,3820.000,17.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522471,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,2015-01-06 00:31:31,7,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522472,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,2016-06-14 07:20:52,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522473,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,2017-02-04 15:31:58,1,1,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1522474,ulfGl4tdbrH05xKzh5lnog,Susanne,2,2011-01-14 00:29:08,0,0,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [62]:
user_df['cool_year'].sort_values(ascending=False).unique()

array([17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
        4.,  3.,  2.,  1., nan])

In [63]:
# describe into 3 decimal places for all the columns consist fo numeric value and describe horizontally
pd.set_option('float_format', lambda x: '%.3f' % x)
user_df.describe().T

,count,mean,std,min,25%,50%,75%,max
review_count,1522476.000,27.481,92.168,0.000,3.000,7.000,20.000,17473.000
friends,1522476.000,50.537,721.834,0.000,1.000,3.000,15.000,206296.000
useful,1522476.000,20.568,459.003,0.000,0.000,0.000,3.000,185823.000
funny,1522476.000,29.136,636.024,0.000,0.000,1.000,4.000,199878.000
elite,1522476.000,1.770,20.509,0.000,0.000,0.000,0.000,12497.000
average_stars,1522476.000,3.693,1.092,1.000,3.090,3.910,4.500,5.000
c_hot,1522476.000,2.195,82.073,0.000,0.000,0.000,0.000,25784.000
c_more,1522476.000,0.352,14.421,0.000,0.000,0.000,0.000,13501.000
c_more,1522476.000,0.218,16.975,0.000,0.000,0.000,0.000,14180.000
c_cute,1522476.000,0.161,12.840,0.000,0.000,0.000,0.000,13654.000
